In [ ]:
!pip install -r /content/drive/MyDrive/project6_2/requirements.txt
!pip install datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import requests
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openai
import json
import torch
from openai import OpenAI
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from warnings import filterwarnings
FutureWarning
filterwarnings('ignore')

class Project6:
    def __init__(self, conf):
        self.conf = conf

        # openai api_key Load 작업
        with open(self.conf['api_file_path'], 'r') as f:
            openai.api_key = f.readline().strip()
            os.environ['OPENAI_API_KEY'] = openai.api_key

        # audio_to_text에서 사용할 client 생성
        self.client_stt = OpenAI()
        self.client_summary = OpenAI()

        # model, tokenizer 불러오기
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = AutoModelForSequenceClassification.from_pretrained(self.conf['model_directory']).to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(self.conf['model_directory'])

    def audio_to_text(self, path):
        # 오디오 파일을 읽어서, 위스퍼를 사용한 변환
        with open(path, 'rb') as audio_file:
            transcript = self.client_stt.audio.transcriptions.create(
                file=audio_file,
                model='whisper-1',
                language='ko',
                response_format='text'
            )

        # 결과 반환
        return transcript

    def text_summary(self, input_txt):
        # 입력데이터를 GPT-3.5-turbo에 전달하고 답변 받아오기
        response = self.client_summary.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": self.conf['summary_system_role']
                },
                {
                    "role": "user",
                    "content": input_txt
                }
            ]
        )

        # 응답 받기
        answer = response.choices[0].message.content

        # 응답형식을 정리하고 return
        return answer

    def predict(self, text):
        # 입력 문장 토크나이징
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        inputs = {key: value.to(self.device) for key, value in inputs.items()}  # 각 텐서를 GPU로 이동

        # 모델 예측
        with torch.no_grad():
            outputs = self.model(**inputs)

        # 로짓을 소프트맥스로 변환하여 확률 계산
        logits = outputs.logits
        probabilities = logits.softmax(dim=1)

        # 가장 높은 확률을 가진 클래스 선택
        pred = torch.argmax(probabilities, dim=-1).item()

        return pred, probabilities

    def proc_pipeline(self, file_paths, progress=True):
        if isinstance(file_paths, list) is False:
            print('filenames should be a list of file name.')
            return False

        result = []
        for idx, path in enumerate(file_paths):
            if progress is True:
                print(f"({idx+1}/{len(file_paths)}) {path}")
            file_name = os.path.basename(path)
            result_stt = self.audio_to_text(path)
            result_summary = self.text_summary(result_stt)
            result_predict = self.predict(result_summary)
            result.append({
                'file_name': file_name,
                'transcript': result_stt,
                'summary': result_summary,
                'predict':result_predict
            })

        return pd.DataFrame(result)



In [ ]:
# Class Project6 생성
conf = {
    'api_file_path': '/content/drive/MyDrive/project6_2/api_key.txt',
    'model_directory': '/content/drive/MyDrive/project6_2/fine_tuned_bert'
}
conf['summary_system_role'] = '''
**역할:** 당신은 숙련된 119 신고 접수 요원입니다. 긴급 출동 팀에게 환자의 상태에 대한 핵심 정보를 명확하고 간결하게 전달해야 합니다. 다음 지침에 따라 신고 내용을 요약해주세요.

1. **환자의 상태:** 환자의 주요 증상, 부상 부위, 의식 상태, 호흡, 맥박, 출혈 여부, 통증 정도 등 긴급 출동 팀이 현장에서 즉각적으로 파악해야 할 중요 정보를 요약합니다.
2. **중요 키워드 추출:** 환자의 상태를 나타내는 중요 키워드를 최대한 많이 추출합니다. 예) 호흡 곤란, 의식 저하, 출혈 심함, 흉통, 복통, 골절 의심 등
3. **요약 형식:** 다음과 같은 형식으로 한 줄로 요약 정보를 출력합니다.

**중요 키워드 : "[키워드1], [키워드2], ...", 내용 요약 : "[요약된 내용]"**

**예시:**

**입력:** "여기 강남역 사거리에서 교통사고가 났어요! 차 두 대가 충돌했고, 한 명이 다친 것 같아요. 빨리 와주세요! 머리가 아프고 다리가 골절된 것 같다고 합니다! 의식은 있는데, 말을 잘 못하고 있어요. 주변에 사람들이 많이 모여있어요. 피를 많이 흘리고 있어요!"

**출력:** 중요 키워드 : "두통, 다리 골절, 의식 있음, 언어능력 저하, 출혈 심함", 내용 요약 : "교통사고 발생, 환자는 두통과 다리 골절 의심, 의식은 있으나 언어능력 저하, 출혈 심함"


**주의 사항:**

* 신고 내용은 다양한 형태로 입력될 수 있습니다. (구어체, 비표준어, 오타 등)
* 최대한 정확하고 간결하게 요약해야 합니다.
* 긴급 출동 팀이 현장에 출동하기 위해 필요한 환자의 상태 정보를 중점적으로 제공해야 합니다.
* 개인정보 (이름, 주소, 연락처 등)는 요약에서 제외합니다.
* 사건의 종류, 발생 장소 및 시간, 주변 상황 등은 요약에 포함하지 않습니다.
'''

project6 = Project6(conf)

In [ ]:
# 추출해야되는 file의 경로+파일명을 list로 전달하면 결과 나옴
audio_path = '/content/drive/MyDrive/project6_2/audio/'
file_names = [audio_path + f for f in os.listdir(audio_path) if os.path.isfile(os.path.join(audio_path, f))]

df = project6.proc_pipeline(file_names)

(1/15) /content/drive/MyDrive/project6_2/audio/audio1.mp3
(2/15) /content/drive/MyDrive/project6_2/audio/audio3.mp3
(3/15) /content/drive/MyDrive/project6_2/audio/audio2.mp3
(4/15) /content/drive/MyDrive/project6_2/audio/audio5.mp3
(5/15) /content/drive/MyDrive/project6_2/audio/audio4.mp3
(6/15) /content/drive/MyDrive/project6_2/audio/B1.mp3
(7/15) /content/drive/MyDrive/project6_2/audio/B2.mp3
(8/15) /content/drive/MyDrive/project6_2/audio/C1.mp3
(9/15) /content/drive/MyDrive/project6_2/audio/A1.mp3
(10/15) /content/drive/MyDrive/project6_2/audio/A2.mp3
(11/15) /content/drive/MyDrive/project6_2/audio/C2.mp3
(12/15) /content/drive/MyDrive/project6_2/audio/D1.mp3
(13/15) /content/drive/MyDrive/project6_2/audio/D2.mp3
(14/15) /content/drive/MyDrive/project6_2/audio/E1.mp3
(15/15) /content/drive/MyDrive/project6_2/audio/E2.mp3


In [ ]:
for idx, row in df.iterrows():
    print(row['file_name'])
    # print(row['transcript'])
    print(row['summary'])
    print(f"예측된 클래스: {row['predict'][0]+1}등급")
    print(f"클래스별 확률: {row['predict'][1]}")
    print()

audio1.mp3
"환자 상태: 머리부상, 의식 있으나 어지러움, 호흡 정상, 맥박 정상, 사건 종류: 넘어짐, 발생 장소 및 시간: 계단, 현재, 주변 상황: 피 나 있음, 응급실 방문 필요"
예측된 클래스: 1등급
클래스별 확률: tensor([[0.5613, 0.1245, 0.2605, 0.0317, 0.0220]])

audio3.mp3
**"환자 상태: 콩으로 인한 코막힘, 사건 종류: 응급사건, 발생 장소 및 시간: 현재 집, 주변 상황: 동생이 힘들어 함"**
예측된 클래스: 3등급
클래스별 확률: tensor([[0.2775, 0.0979, 0.4673, 0.0375, 0.1199]])

audio2.mp3
"환자 상태: 열이 38도, 머리 통증, 어지러움, 사건 종류: 질병, 발생 장소 및 시간: 현재, 주변 상황: 별다른 위험 요소 없음"
예측된 클래스: 3등급
클래스별 확률: tensor([[0.2170, 0.1258, 0.4538, 0.1094, 0.0940]])

audio5.mp3
"환자 상태: 엉덩방아 충격, 꼬리뼈 통증, 사건 종류: 가정 사고, 발생 장소 및 시간: 화장실, 현재, 주변 상황: 응급실 필요성 고민 중"
예측된 클래스: 3등급
클래스별 확률: tensor([[0.3112, 0.0984, 0.4968, 0.0507, 0.0430]])

audio4.mp3
"환자 상태: 머리 충격 후 3시간 경과, 어지러움, 메스꺼움, 사건 종류: 상해, 발생 장소 및 시간: 머리 부딪힘 후 3시간, 주변 상황: 개인 상태, 구체적 조치 미기재"
예측된 클래스: 2등급
클래스별 확률: tensor([[0.2922, 0.3785, 0.2143, 0.0556, 0.0594]])

B1.mp3
"환자 상태: 쓰러진 남성, 경련, 의식 있음, 호흡 있음, 배설물 나옴, 사건 종류: 의료 사고, 발생 장소 및 시간: 고속터미널 지하철역 3번 출구 앞 육교, 현재, 주변 상황: 육교 다리 밑, 7호선 3호선 교통량 있음"
예측된 클래스: 3등급

In [ ]:
ta, tb = project6.predict('가벼운 두통과 메스꺼움을 호소합니다.')
print(f"예측된 클래스: {ta+1}등급")
print(f"클래스별 확률: {tb}")

예측된 클래스: 4등급
클래스별 확률: tensor([[0.0018, 0.0052, 0.0062, 0.9804, 0.0064]])
